In [ ]:
!pip install -q transformers datasets accelerate sentencepiece sacrebleu

!pip install -q indic-nlp-library

!pip install -q pandas tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 13.1 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.6
    Uninstalling transformers-4.57.6:
      Successfully uninstalled transformers-4.57.6


In [ ]:
import json
import pandas as pd
import random
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory


In [ ]:
def load_json_file(path, src_lang, tgt_lang):
    import json
    import pandas as pd

    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    rows = []

    for item in data:
        source = item.get("sourceText") or item.get("sourcetext") or item.get("source") or item.get("source_text")
        target = item.get("targetText") or item.get("targettext") or item.get("target") or item.get("target_text")

        if source is None or target is None:
            continue   # skip broken/empty rows safely

        rows.append({
            "source": source.strip(),
            "target": target.strip(),
            "src_lang": src_lang,
            "tgt_lang": tgt_lang
        })

    df = pd.DataFrame(rows)
    print(f"Loaded {len(df)} rows from {path}")
    return df


In [ ]:
import json

with open("Maithili_Eng.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(data[0])


{'sourceText': 'इंटरव्यू मे ओ ईहो मानलक जे जखन अगिला समर मे सीरीज़ खतम हो जायत, त ओ ‘गेम ऑफ थ्रोन्स’ के ना कोनो प्रीक्वल बनतै, ना सीक्वल। ओ कहलनि जे आब ओ ‘जंग के मैदान आ घोड़ा’ सँ थाक गेल छथि।', 'targetText': "In the interview he also admitted that he wouldn't be doing any Game of Thrones prequels or sequels when the series comes to an end next summer, saying he is 'done with battlefields and horses'."}


In [ ]:
df_mai_hi = load_json_file("maitheli_hindi(Final).json", "mai", "hin")
df_mai_en = load_json_file("Maithili_Eng.json", "mai", "eng")
df_bho_hi = load_json_file("Bhoj_Hindi(Final).json", "bho", "hin")
df_bho_en = load_json_file("Bhojpuri_English.json", "bho", "eng")


Loaded 184 rows from maitheli_hindi(Final).json
Loaded 232 rows from Maithili_Eng.json
Loaded 185 rows from Bhoj_Hindi(Final).json
Loaded 248 rows from Bhojpuri_English.json


In [ ]:
df = pd.concat([df_mai_hi, df_mai_en, df_bho_hi, df_bho_en], ignore_index=True)

print("Total sentences:", len(df))
df.head()

Total sentences: 849


,source,target,src_lang,tgt_lang
0,इंटरव्यू में ओ ईहो मानलक जे जब अगिला समर में स...,इंटरव्यू में उन्होंने यह भी माना कि जब अगले सम...,mai,hin
1,नवंबर से किट सैम शेपर्ड के नाटक “True Well” के...,"नवंबर से, किट सम शेपर्ड की True Well के रिवाइव...",mai,hin
2,अभिनेता हाल में कहलनि कि “गेम ऑफ थ्रोन्स” से न...,अभिनेता ने हाल ही में बताया कि उन्हें अपनी पत्...,mai,hin
3,रोज ई मशहूर फैंटेसी सीरीज़ में यग्रिट के रोल न...,"रोज ने यग्रिट की भूमिका निभाई, जो किट के कैरेक...",mai,hin
4,"ई जोड़ा जून 2018 में स्कॉटलैंड के लेस्ली में, ...",यह जोड़ा जून 2018 में स्कॉटलैंड के लेस्ली के प...,mai,hin


In [ ]:
print("Empty source:", df["source"].isnull().sum())
print("Empty target:", df["target"].isnull().sum())


Empty source: 0
Empty target: 0


In [ ]:
factory = IndicNormalizerFactory()

def normalize_text(text, lang):
    if lang in ["mai", "bho", "hin"]:
        normalizer = factory.get_normalizer("hi")
        return normalizer.normalize(text)
    else:
        return text


In [ ]:
df["source"] = df.apply(lambda x: normalize_text(x["source"], x["src_lang"]), axis=1)
df["target"] = df.apply(lambda x: normalize_text(x["target"], x["tgt_lang"]), axis=1)

print("Normalization completed!")
df.head()


Normalization completed!


,source,target,src_lang,tgt_lang
0,इंटरव्यू में ओ ईहो मानलक जे जब अगिला समर में स...,इंटरव्यू में उन्होंने यह भी माना कि जब अगले सम...,mai,hin
1,"नवंबर से किट सैम शेपर्ड के नाटक ""True Well"" के...","नवंबर से, किट सम शेपर्ड की True Well के रिवाइव...",mai,hin
2,"अभिनेता हाल में कहलनि कि ""गेम ऑफ थ्रोन्स"" से न...",अभिनेता ने हाल ही में बताया कि उन्हें अपनी पत्...,mai,hin
3,रोज ई मशहूर फैंटेसी सीरीज़ में यग्रिट के रोल न...,"रोज ने यग्रिट की भूमिका निभाई, जो किट के कैरेक...",mai,hin
4,"ई जोड़ा जून 2018 में स्कॉटलैंड के लेस्ली में, ...",यह जोड़ा जून 2018 में स्कॉटलैंड के लेस्ली के प...,mai,hin


In [ ]:
df["source"] = df["source"].str.strip()
df["target"] = df["target"].str.strip()


In [ ]:
import unicodedata

def normalize_text(text):
    return unicodedata.normalize("NFC", text)

df["source"] = df["source"].apply(normalize_text)
df["target"] = df["target"].apply(normalize_text)


In [ ]:
df["source"] = df["source"].str.replace(r"\s+", " ", regex=True)
df["target"] = df["target"].str.replace(r"\s+", " ", regex=True)


In [ ]:

before = len(df)
df = df[(df["source"].str.len() > 0) & (df["target"].str.len() > 0)]
print("Removed empty rows:", before - len(df))


Removed empty rows: 0


In [ ]:
before = len(df)
df = df.drop_duplicates(subset=["source", "target", "src_lang", "tgt_lang"])
print("Removed duplicate rows:", before - len(df))


Removed duplicate rows: 0


In [ ]:

groups = df.groupby("target")

unique_targets = list(groups.groups.keys())
print("Unique target sentences:", len(unique_targets))


Unique target sentences: 542


In [ ]:
import random
random.seed(42)

random.shuffle(unique_targets)

n = len(unique_targets)

train_targets = set(unique_targets[:int(0.8 * n)])
val_targets   = set(unique_targets[int(0.8 * n):int(0.9 * n)])
test_targets  = set(unique_targets[int(0.9 * n):])


In [ ]:
def assign_split(row):
    if row["target"] in train_targets:
        return "train"
    elif row["target"] in val_targets:
        return "val"
    else:
        return "test"

df["split"] = df.apply(assign_split, axis=1)


In [ ]:
train_df = df[df["split"] == "train"]
val_df   = df[df["split"] == "val"]
test_df  = df[df["split"] == "test"]

print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))


Train size: 678
Validation size: 81
Test size: 90


In [ ]:
train_targets_set = set(train_df["target"])
val_targets_set   = set(val_df["target"])
test_targets_set  = set(test_df["target"])

print("Train ∩ Val overlap:", len(train_targets_set & val_targets_set))
print("Train ∩ Test overlap:", len(train_targets_set & test_targets_set))
print("Val ∩ Test overlap:", len(val_targets_set & test_targets_set))


Train ∩ Val overlap: 0
Train ∩ Test overlap: 0
Val ∩ Test overlap: 0


In [ ]:
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)
print("Saved train.csv, val.csv, test.csv")


Saved train.csv, val.csv, test.csv


In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments
)


In [ ]:
train_df = pd.read_csv("train.csv")
val_df   = pd.read_csv("val.csv")
test_df  = pd.read_csv("test.csv")

print(train_df.head())


                                              source  \
0  अभिनेता हाल में कहलनि कि "गेम ऑफ थ्रोन्स" से न...   
1  रोज ई मशहूर फैंटेसी सीरीज़ में यग्रिट के रोल न...   
2  ई जोड़ा जून 2018 में स्कॉटलैंड के लेस्ली में, ...   
3  चीन ईहो घोषणा कएलक जे ओकर नागरिक में जे लोग HI...   
4  ई जानकारी गुरुवार के दिन ऑनलाइन सामने आयल छल। ...   

                                              target src_lang tgt_lang  split  
0  अभिनेता ने हाल ही में बताया कि उन्हें अपनी पत्...      mai      hin  train  
1  रोज ने यग्रिट की भूमिका निभाई, जो किट के कैरेक...      mai      hin  train  
2  यह जोड़ा जून 2018 में स्कॉटलैंड के लेस्ली के प...      mai      hin  train  
3  चीन ने यह भी घोषणा किया कि अपने नागरिकों में ज...      mai      hin  train  
4  ई जानकारी गुरुवार के ऑनलाइन सामने आइल, जे एगो ...      mai      hin  train  


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)
test_dataset  = Dataset.from_pandas(test_df)


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset   = Dataset.from_pandas(val_df.reset_index(drop=True))


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
       

In [ ]:
print(model.__class__)


<class 'transformers.models.m2m_100.modeling_m2m_100.M2M100ForConditionalGeneration'>


In [ ]:

df = df.dropna(subset=["source", "target"])
df = df[df["source"].str.strip() != ""]
df = df[df["target"].str.strip() != ""]

print("Clean dataset size:", len(df))


Clean dataset size: 849


In [ ]:
print(train_dataset)
print(train_dataset.column_names)


Dataset({
    features: ['source', 'target', 'src_lang', 'tgt_lang', 'split'],
    num_rows: 678
})
['source', 'target', 'src_lang', 'tgt_lang', 'split']


In [ ]:
def preprocess_function(examples):

    lang_map = {
        "mai": "mai_Deva",
        "bho": "bho_Deva",
        "hin": "hin_Deva",
        "eng": "eng_Latn"
    }

    input_ids = []
    attention_masks = []
    labels = []

    for i in range(len(examples["source"])):

        src = examples["source"][i]
        tgt = examples["target"][i]
        src_lang = examples["src_lang"][i]
        tgt_lang = examples["tgt_lang"][i]

        if src is None or tgt is None:
            continue
        if len(src.strip()) == 0 or len(tgt.strip()) == 0:
            continue

        src_code = lang_map[src_lang]
        tgt_code = lang_map[tgt_lang]

        tokenizer.src_lang = src_code
        tokenizer.tgt_lang = tgt_code

        # Tokenize correctly
        model_inputs = tokenizer(
            src,
            text_target=tgt,
            max_length=128,
            truncation=True,
            padding="max_length"
        )

        input_ids.append(model_inputs["input_ids"])
        attention_masks.append(model_inputs["attention_mask"])
        labels.append(model_inputs["labels"])

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "labels": labels
    }


In [ ]:
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_val = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

print("Tokenization successful!")


Map:   0%|          | 0/678 [00:00<?, ? examples/s]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

Tokenization successful!


In [ ]:
print(tokenized_train[0])


{'input_ids': [206829, 44652, 1035, 37154, 3342, 1182, 69, 8079, 248173, 37344, 2350, 9867, 37939, 248108, 1160, 96575, 248186, 3194, 1160, 142915, 6537, 4820, 17181, 4342, 1996, 15017, 54559, 32771, 1160, 129765, 37914, 122852, 16692, 248225, 248995, 37154, 3342, 1182, 4820, 10010, 2018, 7269, 74172, 111461, 483, 3194, 1160, 42695, 386, 27910, 46493, 36913, 10556, 12984, 248225, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
print(train_dataset[676])


{'source': 'कथित रूप से युआन के परिवार आ मार्केट अधिकारी लोग ओकर मौत खातिर मुआवजा पर बातचीत कर रहल बा।', 'target': "Yuan's family and market authorities are reportedly negotiating compensation for his death.", 'src_lang': 'bho', 'tgt_lang': 'eng', 'split': 'train'}


In [ ]:


for param in model.model.encoder.parameters():
    param.requires_grad = False

print(" Encoder frozen")


 Encoder frozen


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)


In [ ]:
training_args = TrainingArguments(
    output_dir="./nllb_multilingual",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    num_train_epochs=3,
    fp16=True,
    optim="adafactor",
    eval_strategy="no",
    logging_steps=50,
    save_steps=1000,
    save_total_limit=2,
    report_to="none"
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

In [ ]:
trainer.train()


Step,Training Loss
50,57.355205
100,45.320630
150,39.376641
200,35.563428
250,35.377534


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=255, training_loss=42.395703304515166, metrics={'train_runtime': 852.3457, 'train_samples_per_second': 2.386, 'train_steps_per_second': 0.299, 'total_flos': 960814151368704.0, 'train_loss': 42.395703304515166, 'epoch': 3.0})

In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
import os

save_path = "/content/drive/MyDrive/trans_model/nllb_mai_bho_finetune"
os.makedirs(save_path, exist_ok=True)

# Save trained model to Drive
trainer.save_model(save_path)

# Save tokenizer to Drive
tokenizer.save_pretrained(save_path)

print("Translation model saved permanently at:", save_path)


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Translation model saved permanently at: /content/drive/MyDrive/trans_model/nllb_mai_bho_finetune


In [ ]:
!ls /content/drive/MyDrive/trans_model/nllb_mai_bho_finetuned


ls: cannot access '/content/drive/MyDrive/trans_model/nllb_mai_bho_finetuned': No such file or directory


In [ ]:
model.save_pretrained("nllb_mai_bho_finetuned")
tokenizer.save_pretrained("nllb_mai_bho_finetuned")


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('nllb_mai_bho_finetuned/tokenizer_config.json',
 'nllb_mai_bho_finetuned/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
model.save_pretrained("nllb_mai_bho_finetuned")
tokenizer.save_pretrained("nllb_mai_bho_finetuned")
import torch

model_path = "nllb_mai_bho_finetuned"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

print(" Fine-tuned model loaded successfully")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


 Fine-tuned model loaded successfully


In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

drive_path = "/content/drive/MyDrive/trans_model/nllb_mai_bho_finetuned"
# Ensure folder exist karta hai
os.makedirs(drive_path, exist_ok=True)
# Save
model.save_pretrained(drive_path)
tokenizer.save_pretrained(drive_path)

print("Model saved to Drive at:", drive_path)


In [ ]:
import torch

text = "हम रोज सबेरे जल्दी उठैत छी आ घर के सारा काम निपटा के खेत दिस जाइत छी। पिताजी अपन बूढ़ शरीर लए कड़ी मेहनत करैत छथि, मुदा हमरा सभ पर कखनहुँ बोझ नहि बनैत छथि। गाम में आजुकाल पढ़ाई पर बहुत जोर देल जा रहल अछि, मुदा गरीब परिवारक बच्चा सभ के स्कूल जाए में बहुत कठिनाई होइत अछि। कई बेर किताब आ कॉपी के अभाव में ओ सभ पढ़ाई छोड़ देइत अछि। सरकार बहुत योजना बनौने अछि, मुदा सही समय पर सही लाभ नहि पहुँचैत अछि। "   # Maithili sentence

# Set languages
tokenizer.src_lang = "mai_Deva"

inputs = tokenizer(text, return_tensors="pt").to(device)

with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids("hin_Deva"),
        max_length=200
    )

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

print(" Input :", text)
print(" Output:", result[0])